In [8]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2020-06-08 04:39:39--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-06-08 04:39:39 (221 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
from tensorflow.keras.callbacks import Callback
class PerfCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > DESIRED_ACCURACY):
      self.model.stop_training = True
callback = PerfCallback()

In [21]:
from tensorflow.keras.optimizers import RMSprop
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
model.compile(loss="binary_crossentropy", optimizer=RMSprop(lr=0.001), metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 32)        18464     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 9248)             

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [23]:
history = model.fit(train_generator, epochs=30, verbose=1, callbacks=[callback])

Epoch 1/30
3/3 [==============================] - 0s 53ms/step - loss: 1.6066 - accuracy: 0.5500
Epoch 2/30
3/3 [==============================] - 0s 25ms/step - loss: 0.6893 - accuracy: 0.5000
Epoch 3/30
3/3 [==============================] - 0s 27ms/step - loss: 0.6561 - accuracy: 0.7000
Epoch 4/30
3/3 [==============================] - 0s 25ms/step - loss: 0.5825 - accuracy: 0.8500
Epoch 5/30
3/3 [==============================] - 0s 29ms/step - loss: 0.6482 - accuracy: 0.5250
Epoch 6/30
3/3 [==============================] - 0s 25ms/step - loss: 0.3941 - accuracy: 0.7875
Epoch 7/30
3/3 [==============================] - 0s 27ms/step - loss: 0.4554 - accuracy: 0.7625
Epoch 8/30
3/3 [==============================] - 0s 26ms/step - loss: 0.2012 - accuracy: 0.9250
Epoch 9/30
3/3 [==============================] - 0s 28ms/step - loss: 0.1589 - accuracy: 0.9625
Epoch 10/30
3/3 [==============================] - 0s 36ms/step - loss: 0.1327 - accuracy: 0.9625
Epoch 11/30
3/3 [============